# Évaluation qualitative
> Comparons les différents algorithmes en terme de qualité de prédiction et de temps de calcul

# Comparaison entre algorithmes

Charger les bibliothèques et les données

In [50]:
%autosave 0
%reload_ext autoreload
%autoreload 2

from data import *
import popularity
import knn
import knn2
import svd
#import als

from eval import *

Autosave disabled


In [51]:
import sys
sys.path
import data

In [52]:
M = load_data(tiny=True)
M.shape, np.sum(~np.isnan(M))

((50, 40), 389)

### User-based collaborative filtering (knn)

Liste d'algorithmes à comparer

In [53]:
k_list = [1, 3, 5, 7, 9, 10, 15, 20]
recommenders = [
    {"fn": popularity.complete, "label": "popularity"},
    {"fn": lambda M_train: knn.complete(M_train, k=1), "label": f"knn1"},
    {"fn": lambda M_train: knn.complete(M_train, k=3), "label": f"knn3"},
    {"fn": lambda M_train: knn.complete(M_train, k=5), "label": f"knn5"},
    {"fn": lambda M_train: knn.complete(M_train, k=7), "label": f"knn7"},
    {"fn": lambda M_train: knn.complete(M_train, k=9), "label": f"knn9"},
    {"fn": lambda M_train: knn.complete(M_train, k=10), "label": f"knn10"},
    {"fn": lambda M_train: knn.complete(M_train, k=15), "label": f"knn15"},
    {"fn": lambda M_train: knn.complete(M_train, k=20), "label": f"knn20"},
    {"fn": lambda M_train: knn.complete(M_train, k=25), "label": f"knn25"},
    {"fn": lambda M_train: knn.complete(M_train, k=30), "label": f"knn30"},
    {"fn": lambda M_train: knn.complete(M_train, k=40), "label": f"knn40"},
    {"fn": lambda M_train: knn.complete(M_train, k=50), "label": f"knn50"}
]

# recommenders += [{"fn": lambda M_train: knn.complete(M_train, k=k), "label": f"knn_{k}"} for k in k_list]

Comparaison

In [54]:
user_based_knn_leaderboard = quantitative_comparison(RMSE, M, recommenders, prop=0.8, nrep=3)
user_based_knn_leaderboard

/home/ensai/Documents/Cours_ENSAI_3A/Rec_systems/TP/repo/mine/popularity.py:40: RuntimeWarning: Mean of empty slice
  scores = np.nanmean(M_train, axis=0)    # compute the average rating for each movies


,recommender,validation score,training score,computation time
0,popularity,1.312026,0.000000,0.000455
1,knn1,1.337462,1.631381,0.675541
2,knn3,1.213324,1.142317,0.739217
3,knn5,1.211400,0.987500,0.814083
4,knn7,1.191549,0.919205,0.787700
5,knn9,1.184549,0.872570,1.252703
6,knn10,1.173776,0.852622,0.749827
7,knn15,1.157693,0.800289,0.805220
8,knn20,1.154527,0.786265,0.748891
9,knn25,1.148655,0.783178,0.630211


La meilleure valeur de k est 25. Elle permet d'obtenir le meilleur score sur l'échantillon de validation (1.148655).

### Item-based collaborative filtering (knn)

Liste d'algorithmes à comparer

In [55]:
recommenders = [
    {"fn": lambda M_train: knn2.complete(M_train, k=1), "label": f"knn2_1"},
    {"fn": lambda M_train: knn2.complete(M_train, k=3), "label": f"knn2_3"},
    {"fn": lambda M_train: knn2.complete(M_train, k=5), "label": f"knn2_5"},
    {"fn": lambda M_train: knn2.complete(M_train, k=7), "label": f"knn2_7"},
    {"fn": lambda M_train: knn2.complete(M_train, k=9), "label": f"knn2_9"},
    {"fn": lambda M_train: knn2.complete(M_train, k=10), "label": f"knn2_10"},
    {"fn": lambda M_train: knn2.complete(M_train, k=15), "label": f"knn2_15"},
    {"fn": lambda M_train: knn2.complete(M_train, k=20), "label": f"knn2_20"},
    {"fn": lambda M_train: knn2.complete(M_train, k=25), "label": f"knn2_25"},
    {"fn": lambda M_train: knn2.complete(M_train, k=30), "label": f"knn2_30"}
]

# recommenders = [{"fn": lambda M_train: knn2.complete(M_train, k=k), "label": f"knn_{k}"} for k in k_list]

In [56]:
item_based_knn_leaderboard = quantitative_comparison(RMSE, M[:, 1:], recommenders, prop=0.8, nrep=3)
item_based_knn_leaderboard

,recommender,validation score,training score,computation time
0,knn2_1,1.203240,1.569281,0.581577
1,knn2_3,1.059928,1.192100,0.624341
2,knn2_5,1.041348,1.041334,0.704241
3,knn2_7,1.038990,0.959289,0.665937
4,knn2_9,1.044782,0.897705,0.685661
5,knn2_10,1.046224,0.870014,0.730516
6,knn2_15,1.045143,0.799300,0.678415
7,knn2_20,1.048382,0.769467,0.640338
8,knn2_25,1.040307,0.765439,0.672479
9,knn2_30,1.042131,0.767008,0.665543


La première colonne (film) des données est pleine de `nan`. Par conséquent, même après avoir appliqué la fonction `knn2.complete()`, elle reste inchangée et empêche le calcul des scores de validation. Je supprime donc la colonne concernée avant d'appliquer les modèles basés sur l'approche item-based. 

En outre, la meilleure valeur de k ici est 7, avec un score de validation de 1,03899.

# Paramétrage de méthode basée sur SVD

Charger les bibliothèques et les données

In [74]:
M = load_data(tiny=False)
M.shape, np.sum(~np.isnan(M))

((500, 400), 31553)

On fait varier $k$. Observe-t-on un phénomène de sur-apprentissage ?

In [75]:
recommenders = [
#    {"fn": lambda M_train: knn.complete(M_train, k=10), "label": "knn10"},
    {"fn": lambda M_train: svd.complete(M_train, k=3), "label": "svd3"},
    {"fn": lambda M_train: svd.complete(M_train, k=5), "label": "svd5"},
    {"fn": lambda M_train: svd.complete(M_train, k=8), "label": "svd8"},
    {"fn": lambda M_train: svd.complete(M_train, k=10), "label": "svd10"},
    {"fn": lambda M_train: svd.complete(M_train, k=20), "label": "svd20"},
    {"fn": lambda M_train: svd.complete(M_train, k=50), "label": "svd50"},
    {"fn": lambda M_train: svd.complete(M_train, k=100), "label": "svd100"},
    {"fn": lambda M_train: svd.complete(M_train, k=200), "label": "svd200"},
    {"fn": lambda M_train: svd.complete(M_train, k=400), "label": "svd400"},
    ]
res_fill_with_zeros = quantitative_comparison(RMSE, M, recommenders, prop=0.8, nrep=4)
res_fill_with_zeros

,recommender,validation score,training score,computation time
0,svd3,2.651221,2.574239e+00,0.208200
1,svd5,2.575149,2.453971e+00,0.139622
2,svd8,2.553775,2.337218e+00,0.127798
3,svd10,2.570568,2.283717e+00,0.168516
4,svd20,2.716986,2.076845e+00,0.129136
5,svd50,3.148233,1.605829e+00,0.191183
6,svd100,3.532771,1.038684e+00,0.195464
7,svd200,3.781206,3.800365e-01,0.144218
8,svd400,3.828862,7.880122e-15,0.132540


Oui, on observe ici un phénomène de sur-apprentissage car le score sur les données d'entrainement dimunue jusqu'à atteindre presque 0 (pour k=400) alors que le score sur les données de validation augmente.

La meilleure valeur de k est 8 avec un score de validation de 2.5537

Quelques recommendations faites par cette méthode

In [80]:
svd.recommend(M, 6, k=8)

58

In [81]:
svd.recommend(M, 155, k=8)

127

In [82]:
svd.recommend(M, 296, k=8)

172

In [83]:
svd.recommend(M, 361, k=8)

272